In [ ]:
!pip install ray==0.8.4
#!pip install ray==1.0.0

In [ ]:
!pip install dm-tree==0.1.5

In [ ]:
!conda install opencv

#!pip install opencv_python-4.4.0.44-cp37-cp37m-manylinux2014_x86_64.whl

In [ ]:
import cv2

In [ ]:
!pip install torch==1.6.0

In [ ]:
!pip install gym==0.17.3

In [ ]:
!pip install ibm-cos-sdk

In [ ]:
!pip install boto3

In [ ]:
!pip install tensorflow==2.3.1

In [1]:
import ray
import gym
import tensorflow as tf


In [2]:
ray.__version__

'0.8.4'

In [3]:
gym.__version__

'0.17.3'

In [4]:
tf.__version__

'2.3.1'

In [5]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0



def is_watson_studio():
    import os
    return 'wsuser' in os.getcwd()


def load_market_data (file_name):
    


    if is_watson_studio():
        
        client_244519148910498e82791851c5dd0328 = ibm_boto3.client(service_name='s3',
        ibm_api_key_id='B4M4KvSyuL9a46802UJTEu2i5YqsKLu5JTk1bDSdiw40',
        ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
        config=Config(signature_version='oauth'),
        endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')
        body = client_244519148910498e82791851c5dd0328.get_object(Bucket='degray12102020-donotdelete-pr-44gg35tcgd2qty',Key=file_name)['Body']
        # add missing __iter__ method, so pandas accepts body as file-like object
        if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
        df_data_1 = pd.read_csv(body)

    else:
        df_data_1 = pd.read_csv(file_name)
        
    return df_data_1


In [6]:
assets = ['DJI', 'BTC-USD', 'IBM', 'VOW.DE']
df = None
for asset in assets:
    df_asset = load_market_data(f'{asset}.csv')
    df_asset = df_asset[['Date','Close']]
    df_asset.columns = ['date', asset]
    if df is None:
        df = df_asset
    else:
        df = pd.merge(df,df_asset, on='date')

In [7]:
df = df.dropna()
df

,date,DJI,BTC-USD,IBM,VOW.DE
0,2014-09-17,17156.849609,457.334015,192.800003,173.300003
1,2014-09-18,17265.990234,424.440002,193.750000,174.300003
2,2014-09-19,17279.740234,394.795990,194.000000,174.149994
3,2014-09-22,17172.679688,402.152008,193.110001,172.750000
4,2014-09-23,17055.869141,435.790985,191.619995,169.149994
...,...,...,...,...,...
1496,2020-10-08,28425.509766,10923.627930,131.490005,147.000000
1497,2020-10-09,28586.900391,11079.465820,127.790001,146.699997
1498,2020-10-12,28837.519531,11555.363281,127.209999,149.100006
1499,2020-10-13,28679.810547,11425.899414,125.099998,147.199997


In [8]:
#df['ts']=pd.to_datetime(df.date)

In [9]:
#(pd.to_datetime(df['ts']).astype(int) / 1000 / 1000 / 60 / 60 / 24 / 1000).astype(int)

In [10]:
n_days = len(df)
min_list = []
max_list = []

for i in range(n_days):
    row = df.iloc[i, 2:]
    min_list.append(min(row))
    max_list.append(max(row))
    
print("{:5.2f}% worst case annualized".format(sum(min_list) / n_days / 365))
print("{:5.2f}% best case annualized".format(sum(max_list) / n_days / 365))

 0.38% worst case annualized
12.25% best case annualized


In [11]:
import gym
from gym.spaces import Discrete, Box
from gym.utils import seeding
import numpy as np
import random

In [12]:
class MarketBandit (gym.Env):
    
    def __init__ (self, config={}):
        self.max_inflation = np.array([[100.]])
        self.assets = assets

        #0: do nothing, 1-4 buy, 5-9 sell for amount x
        self.action_space = Discrete(9)
        self.observation_space = Box(-self.max_inflation,self.max_inflation)
        self.df = df
        self.portfolio =  dict.fromkeys(self.assets, 0.0)
        self.trade_amount_per_step = 100.
        self.money_invested = 0


    def reset (self):
        self.index = 0
        self.done = False
        self.info = {}

        return np.array([[0.]])

    def get_portfolio_value(self,row):
        portfolio_sum = 0.
        for asset in self.assets:
            portfolio_sum += self.portfolio[asset]*row[asset]
        return portfolio_sum

    def step (self, action):
        if self.done:
            reward = 0.
            regret = 0.
        else:
            row = self.df.iloc[self.index]

            portfolio_value_before = self.get_portfolio_value(row)
            
            if action == 0:
                reward = 0.
                regret = 0
            else:
                buy = False
                
                if action < 5: # buy
                    buy = True
                    asset_to_trade = self.assets[action-1]
                else: # sell
                    asset_to_trade = self.assets[action-5]
                    
                price = row[asset_to_trade]
                asset_to_trade_fraction = self.trade_amount_per_step / price
                
                if buy:
                    self.portfolio[asset_to_trade]+=asset_to_trade_fraction
                    self.money_invested += self.trade_amount_per_step
                else:
                    self.portfolio[asset_to_trade]+=asset_to_trade_fraction

            portfolio_value_after = self.get_portfolio_value(row)

            reward = float(portfolio_value_after-portfolio_value_before) / 20000

            self.index += 1

            if self.index >= len(self.df):
                self.done = True

        #context = [self.cur_context]
        #context = self.observation_space.sample()

        self.info = {
            "regret": 0.,
            "index": self.index
        }
         
        return [np.array([[0.]]), reward, self.done, self.info]


    def seed (self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

In [13]:
bandit = MarketBandit()
bandit.reset()

for i in range(len(df)):
    action = bandit.action_space.sample()
    obs = bandit.step(action)
    print(action, obs)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


8 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 1}]
5 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 2}]
5 [array([[0.]]), 0.005000000000000002, False, {'regret': 0.0, 'index': 3}]
0 [array([[0.]]), 0.0, False, {'regret': 0.0, 'index': 4}]
3 [array([[0.]]), 0.005000000000000003, False, {'regret': 0.0, 'index': 5}]
5 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 6}]
5 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 7}]
1 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 8}]
5 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 9}]
1 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 10}]
5 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 11}]
0 [array([[0.]]), 0.0, False, {'regret': 0.0, 'index': 12}]
7 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 13}]
2 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 14}]
2 [array([[0.]]), 0.005, False, {'regret': 0.0, 'index': 15}]
2 [array([[0.]]), 0.005, False, {'regret': 0.0, 'inde

We can use this environment in a kind of *monte carlo simulation* to measure a baseline for what the rewards would be over a long period if you always used a random action.

In [14]:
done = 1
reward_list = []
iterations = 10000 #50000

for i in range(iterations):
    if done == 1:
        bandit.reset()

    action = bandit.action_space.sample()
    obs = bandit.step(action)
    context, reward, done, info = obs
    reward_list.append(reward)
    print(action, context, reward, done, info)

0 [[0.]] 0.0 False {'regret': 0.0, 'index': 1}
2 [[0.]] 0.005 False {'regret': 0.0, 'index': 2}
3 [[0.]] 0.005 False {'regret': 0.0, 'index': 3}
5 [[0.]] 0.005 False {'regret': 0.0, 'index': 4}
0 [[0.]] 0.0 False {'regret': 0.0, 'index': 5}
7 [[0.]] 0.005 False {'regret': 0.0, 'index': 6}
8 [[0.]] 0.005 False {'regret': 0.0, 'index': 7}
3 [[0.]] 0.005 False {'regret': 0.0, 'index': 8}
8 [[0.]] 0.005 False {'regret': 0.0, 'index': 9}
1 [[0.]] 0.005 False {'regret': 0.0, 'index': 10}
3 [[0.]] 0.005 False {'regret': 0.0, 'index': 11}
0 [[0.]] 0.0 False {'regret': 0.0, 'index': 12}
5 [[0.]] 0.005 False {'regret': 0.0, 'index': 13}
0 [[0.]] 0.0 False {'regret': 0.0, 'index': 14}
0 [[0.]] 0.0 False {'regret': 0.0, 'index': 15}
1 [[0.]] 0.005 False {'regret': 0.0, 'index': 16}
5 [[0.]] 0.005 False {'regret': 0.0, 'index': 17}
0 [[0.]] 0.0 False {'regret': 0.0, 'index': 18}
0 [[0.]] 0.0 False {'regret': 0.0, 'index': 19}
5 [[0.]] 0.005 False {'regret': 0.0, 'index': 20}
5 [[0.]] 0.005 False {'

In [ ]:
import ray
from ray.rllib.agents.trainer import with_base_config, with_common_config
from ray.rllib.contrib.bandits.agents.lin_ts import TS_CONFIG
from ray.rllib.contrib.bandits.agents.lin_ts import LinTSTrainer


In [15]:
ray.init(temp_dir='.', memory=52428800, object_store_memory=78643200, num_cpus=1, num_gpus=0, local_mode=True)

{}

We need a custom config object with our parameters for `MarketBandit`. We do this building on the default `TS_CONFIG` object for _LinTS_:

In [ ]:
market_config = with_base_config(TS_CONFIG, {
    "env":           MarketBandit
})

stop = {
    "training_iteration": 100
}

Also, we'll define a custom trainer, which builds on the `LinTSTrainer`, with "updates". Note that it's the first argument we'll pass to `tune.run()` in the following cell. When all we need is `LinTSTrainer`, as is, and no extra custom config settings, we can just pass the string `contrib/LinTS` to `tune.run()`.  

In [ ]:
MarketLinTSTrainer = LinTSTrainer.with_updates(
    name="MarketLinTSTrainer",
    default_config=market_config,      # Will be merged with Trainer.COMMON_CONFIG (rllib/agent/trainer.py)
    #default_policy=[somePolicyClass]  # If we had a policy...
)

In [ ]:
analysis = ray.tune.run(
    MarketLinTSTrainer,
    config=market_config,
    stop=stop,
    num_samples=3,    
    checkpoint_at_end=True,
    verbose=1
)

In [ ]:
!cat /home/jovyan/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_0_2020-10-16_15-57-0390q7c64v/error.txt

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

In [16]:
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

2020-10-19 08:23:23,521	WARNING compression.py:16 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.


In [17]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["sgd_minibatch_size"] = 64
trainer_config["num_sgd_iter"] = 10

In [18]:
trainer = PPOTrainer(trainer_config, MarketBandit)
print(f'Training iterations: ', end='')
for i in range(20):
    print('.', end='')
    trainer.train()
print('done')


2020-10-19 08:23:27,332	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-10-19 08:23:27,344	ERROR logger.py:193 -- pip install 'ray[tune]' to see TensorBoard files.
2020-10-19 08:23:27,345	WARNING logger.py:308 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.
2020-10-19 08:23:27,346	ERROR syncer.py:39 -- Log sync requires rsync to be installed.
2020-10-19 08:23:27,353	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-10-19 08:23:31,984	INFO trainable.py:217 -- Getting current IP.
2020-10-19 08:23:31,986	WARNING util.py:37 -- Install gputil for GPU system monitoring.


Training iterations: ....................done


In [19]:
env = MarketBandit()
state = env.reset()

done = False
max_state = -1
cumulative_reward = 0

while not done:
    action = trainer.compute_action(state)
    state, reward, done, results = env.step(action)
    max_state = max(max_state, state)
    cumulative_reward += reward

print(f'Cumulative reward you received is: {cumulative_reward*20000}. Congratulations!')
print(f'Portfolio value: {env.get_portfolio_value(df.iloc[len(df)-1])}')
print(f'Money invested {env.money_invested}')

Cumulative reward you received is: 147199.99999999767. Congratulations!
Portfolio value: 583741.1762928049
Money invested 54900.0


## Analyzing the results

Let's analyze the rewards and cumulative regrets of these trials.

In [ ]:
df_ts = pd.DataFrame()

for key, df_trial in analysis.trial_dataframes.items():
    df_ts = df_ts.append(df_trial, ignore_index=True)
    
df_ts.head()

In [ ]:
rewards = df_ts \
    .groupby("info/num_steps_trained")["episode_reward_mean"] \
    .aggregate(["mean", "max", "min", "std"])

rewards

In [ ]:
plot_line_with_stddev(rewards, x_col='info/num_steps_trained', y_col='mean', stddev_col='std', 
                      title='Rewards vs. Steps', x_axis_label='step', y_axis_label='reward')

([image](../../images/rllib/Market-Bandit-Rewards-vs-Steps.png))

The rewards reach what appears to be nearly optimal by 3000 steps, then shows some slow improvement beyond 8000.

In [ ]:
regrets = df_ts \
    .groupby("info/num_steps_trained")["info/learner/default_policy/cumulative_regret"] \
    .aggregate(["mean", "max", "min", "std"])

regrets

In [ ]:
plot_cumulative_regret(regrets)

([image](../../images/rllib/Market-Bandit-Cumulative-Regret.png))

## Evaluating the Trained Policy

Overall, how well did the trained policy perform? The results should be better than random, but less than the best case.

In [ ]:
print("{:5.2f}% optimized return annualized".format(max(rewards["mean"]) / n_years))

You should see a number between 6-8%. That's better than the random action baseline of 3.75%, but no where near the best case scenario of 15.18% return. Hence, our regrets continue to grow over time...

Note that investing solely in the S&P stock index which would have produced better than 8% return over that period -- that is, if one could wait 92 years. However, investing one's entire portfolio into stocks can become quite a risky policy in the short-term, so we were exploring how to balance a portfolio given only limited information.

In any case, the contextual bandit performed well considering that it could only use *inflation* for the context of its decisions, and could only take actions once each year.

## Exercise 1

Try using a `LinUCBTrainer`-based trainer. How does the annualized return compare?

---

## Extra - Restoring from a Checkpoint

In the previous lesson, [05 Thompson Sampling](05-Thompson-Sampling.ipynb), we showed how to restore a trainer from a checkpoint, but almost "in passing". Let's use this feature again, this time with our custom trainer class `MarketLinTSTrainer`.

In [ ]:
trial = analysis.trials[0]
path = trial.checkpoint.value
print(f'checkpoint_path: {path}')

In [ ]:
trainer = MarketLinTSTrainer(market_config)  # create instance and then restore from checkpoint
trainer.restore(path)

Access the model, to review the distribution of arm weights

In [ ]:
model = trainer.get_policy().model
means = [model.arms[i].theta.numpy() for i in range(3)]
covs = [model.arms[i].covariance.numpy() for i in range(3)]
means, covs, model.arms[0].theta.numpy()

A final note: when you checkpoint it will change how the training performs in this notebook, if you rerun the training! So be sure to start from scratch when doing experiments here, if that's what you intend!

In [ ]:
ray.shutdown()  # "Undo ray.init()".